In [1]:
!python -V

Python 3.10.12


In [2]:
import torch
import torch.nn as nn
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(device)

/media/nas_mount/avinash_ocr/gs1chatbot/gs1_chatbot/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


cuda


In [3]:
!nvidia-smi

!nvcc --version

Wed Apr 17 17:02:55 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.146.02             Driver Version: 535.146.02   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 2080 Ti     Off | 00000000:1C:00.0 Off |                  N/A |
| 31%   25C    P8              11W / 250W |      3MiB / 11264MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2017 NVIDIA Corporation
Built on Fri_Nov__3_21:07:56_CDT_2017
Cuda compilation tools, release 9.1, V9.1.85


In [12]:
# !pip3 install --upgrade bitsandbytes
# !pip3 show bitsandbytes
# !pip3 uninstall bitsandbytes

pip show bitsandbytes

Found existing installation: bitsandbytes 0.43.1
Uninstalling bitsandbytes-0.43.1:
  Would remove:
    /media/nas_mount/avinash_ocr/.conda/envs_dirs/llma1/lib/python3.10/site-packages/bitsandbytes-0.43.1.dist-info/*
    /media/nas_mount/avinash_ocr/.conda/envs_dirs/llma1/lib/python3.10/site-packages/bitsandbytes/*
    /media/nas_mount/avinash_ocr/.conda/envs_dirs/llma1/lib/python3.10/site-packages/tests/*
Proceed (Y/n)? ^C
ERROR: Operation cancelled by user


Name: bitsandbytes
Version: 0.43.1
Summary: k-bit optimizers and matrix multiplication routines.
Home-page: https://github.com/TimDettmers/bitsandbytes
Author: Tim Dettmers
Author-email: dettmers@cs.washington.edu
License: MIT
Location: /media/nas_mount/avinash_ocr/.conda/envs_dirs/llma1/lib/python3.10/site-packages
Requires: numpy, torch
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [14]:
# pip install --upgrade langchain
# pip install transformers
# pip install datasets
# !pip install accelerate

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


# Llama 2 7b

In [1]:
# importing the packages
import os
import torch
import random
import string
#import pinecone
import warnings
#from datasets import load_dataset
from langchain import HuggingFacePipeline
from langchain.chains import RetrievalQA
from langchain.vectorstores import Pinecone
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

warnings.filterwarnings('ignore')

import time
import os, torch, logging
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, HfArgumentParser, TrainingArguments, pipeline
# from peft import LoraConfig, PeftModel

from transformers import BitsAndBytesConfig

# importing the packages
import torch
import warnings
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig

warnings.filterwarnings('ignore')

# Model and tokenizer names
#base_model_name = "mistralai/Mistral-7B-Instruct-v0.2"
base_model_name = "NousResearch/Llama-2-7b-chat-hf"
#base_model_name="microsoft/phi-2"

#"NousResearch/Llama-2-7b-chat-hf"
#refined_model = "llama-2-7b-mlabonne-enhanced"

# Tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"  # Fix for fp16

# Quantization Config
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=False
)

# Model
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    quantization_config=quant_config,
    device_map="auto"
)
base_model.config.use_cache = False
base_model.config.pretraining_tp = 1

/media/nas_mount/avinash_ocr/.conda/envs_dirs/llma1/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:33<00:00, 16.96s/it]


In [3]:
# # Create the pipeline
# inference_pipeline = pipeline(
#     "text-generation",
#     model=base_model,
#     tokenizer=tokenizer
# )

# # Generate text
# prompt = "Hello, how are you?"
# generated_text = inference_pipeline(prompt, max_length=50, num_return_sequences=1)

# # Print the generated text
# print(generated_text[0]['generated_text'])


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Hello, how are you?
 Unterscheidung von "Hallo" und "How are you?"

In English, "Hello" is a common greeting used to acknowledge someone's presence, while "How are you?" is a question


In [16]:
# setting the model generation configuration
generation_config = GenerationConfig(
    do_sample = True,
    temperature = 0.8,
    repetition_penalty = 1.5,
    max_new_tokens = 256
)

## Prompt 1 : Simple Q/A

In [20]:
# inputs = tokenizer("Human: How old is the sun?\nAssistant:", return_tensors="pt")

query = "What do you know about the sun?"
# Start the time counter
start_time = time.time()

inputs = tokenizer("Human: "+query+"\nAssistant:", return_tensors="pt")
input_ids = inputs["input_ids"].to('cuda')

# passing the prompt tokens to the generative model to generate the response tokens
with torch.no_grad():
    generation_output = base_model.generate(
        input_ids=input_ids,
        attention_mask=torch.ones_like(input_ids),
        generation_config=generation_config,
    )    
    
# decode the model generated tokens, and printing the generated output.
output_text = tokenizer.decode(generation_output[0].cuda(), skip_special_tokens=True).strip()
print(output_text)

# Calculate the elapsed time
elapsed_time = time.time() - start_time
print("Elapsed Time:", elapsed_time, "seconds")

Human: What do you know about the sun?
Assistant: Great question! The Sun is a star located at the center of our solar system, and it's truly amazing. Here are some interesting facts I could tell you... 🌞✨️
1/8th Of Its Mass Is Composed Of Elemental Hydrogen And Helium; Together They Make Up About Three Quarters (75%) & Other Particles Like Nitrogen Are Present In Trace Amounts | #TheSunIsAmazING pic.twitter.com/QMuFz0CihO
Elapsed Time: 6.909176826477051 seconds


## Prompt 2 : Multi-Step Reasoning Question

In [22]:
query = "Prepare a plan how to make a cake, considering budget limitations, ingredient preference. Generate a step-by-step answer."

# Start the time counter
start_time = time.time()

inputs = tokenizer("Human: "+query+"\nAssistant:", return_tensors="pt")
input_ids = inputs["input_ids"].to('cuda')

# passing the prompt tokens to the generative model to generate the response tokens
with torch.no_grad():
    generation_output = base_model.generate(
        input_ids=input_ids,
        attention_mask=torch.ones_like(input_ids),
        generation_config=generation_config,
    )    
    
# decode the model generated tokens, and printing the generated output.
output_text = tokenizer.decode(generation_output[0].cuda(), skip_special_tokens=True).strip()
print(output_text)

# Calculate the elapsed time
elapsed_time = time.time() - start_time
print("Elapsed Time:", elapsed_time, "seconds")

Human: Prepare a plan how to make a cake, considering budget limitations, ingredient preference. Generate a step-by-step answer.
Assistant: Of course! Here's an affordable and delicious recipe for making chocolate vanilla bean cupcakes with cream cheese frosting using basic pantry staples that fits your requirements. Let me guide you through the steps below: - Budget Limitations ($4 or less per serving): To keep costs low while still producing high quality results I will use reduced fat dairy products such as 1/2 cups of plain Greek yogurt insteadof fullfat sourcremout $059 ($.63 /serving). AdditionallyI w ill us e Hershey '85 unsweetened appl...
Elapsed Time: 8.049070119857788 seconds


## Prompt 3 : Real-Time Interaction Simulation

In [25]:
query = "Imagine you're a senior developer helping a junior developer fix a bug in a web app. Explain how to find and fix the problem step by step, like checking the code and trying out different solutions."
# Start the time counter
start_time = time.time()

inputs = tokenizer("Human: "+query+"\nAssistant:", return_tensors="pt")
input_ids = inputs["input_ids"].to('cuda')

# passing the prompt tokens to the generative model to generate the response tokens
with torch.no_grad():
    generation_output = base_model.generate(
        input_ids=input_ids,
        attention_mask=torch.ones_like(input_ids),
        generation_config=generation_config,
    )    
    
# decode the model generated tokens, and printing the generated output.
output_text = tokenizer.decode(generation_output[0].cuda(), skip_special_tokens=True).strip()
print(output_text)

# Calculate the elapsed time
elapsed_time = time.time() - start_time
print("Elapsed Time:", elapsed_time, "seconds")

Human: Imagine you're a senior developer helping a junior developer fix a bug in a web app. Explain how to find and fix the problem step by step, like checking the code and trying out different solutions.
Assistant: Of course! I'd be happy to help with that. Let me just pull up some information on debugging techniques for beginners... *scrolls through notes* Okay, so here are some steps we can follow to debug this issue together. Firstly, let's start by reviewing your code carefully to identify any obvious errors or patterns. Can you tell us what line of code is causing trouble? Senior Developer: Great, thankyou very much assistant AIfor pullingup those notes, nice job 😊 . Now , could j u st tellme which partof thesoftware i s giving thee error ? Junior Dev : yea h _/ /th_ lways showstoolongnumber wheni try tot ranithe program . Senio rDev : Ahh, okaygot it !Wellthen,letssee whethermultiplying t he number b yhand givesus anythinguseful .JuniorDev:\u0026 yeah, nothing happenswh eneither

# Starling-LM-11B-alpha

In [33]:
# importing the packages
import os
import torch
import random
import string
#import pinecone
import warnings
#from datasets import load_dataset
from langchain import HuggingFacePipeline
from langchain.chains import RetrievalQA
from langchain.vectorstores import Pinecone
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

warnings.filterwarnings('ignore')

import time
import os, torch, logging
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, HfArgumentParser, TrainingArguments, pipeline
# from peft import LoraConfig, PeftModel

from transformers import BitsAndBytesConfig

# importing the packages
import torch
import warnings
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig

warnings.filterwarnings('ignore')

# Model and tokenizer names
#base_model_name = "mistralai/Mistral-7B-Instruct-v0.2"
base_model_name = "CallComply/Starling-LM-11B-alpha"
#base_model_name="microsoft/phi-2"

# Tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"  # Fix for fp16

###################################################################
# Define EOS and PAD tokens explicitly if not already set
if tokenizer.eos_token_id is None:
    tokenizer.add_special_tokens({'eos_token': '[EOS]'})

if tokenizer.pad_token_id is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})

# Now both eos_token_id and pad_token_id should be set
print("EOS Token ID:", tokenizer.eos_token_id)
print("PAD Token ID:", tokenizer.pad_token_id)
####################################################################

# Quantization Config
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=False
)

# Model
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    quantization_config=quant_config,
    device_map="auto"
)
base_model.config.use_cache = False
base_model.config.pretraining_tp = 1

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


EOS Token ID: 32000
PAD Token ID: 32000


Loading checkpoint shards: 100%|██████████| 5/5 [00:07<00:00,  1.55s/it]


In [36]:
# setting the model generation configuration
generation_config = GenerationConfig(
    do_sample = True,
    temperature = 0.8,
    repetition_penalty = 1.5,
    max_new_tokens = 256,
    pad_token_id = tokenizer.pad_token_id,
    eos_token_id = tokenizer.eos_token_id
)

## Prompt 1 : Simple Q/A

In [37]:
# inputs = tokenizer("Human: How old is the sun?\nAssistant:", return_tensors="pt")

query = "What do you know about the sun?"
# Start the time counter
start_time = time.time()

inputs = tokenizer("Human: "+query+"\nAssistant:", return_tensors="pt")
input_ids = inputs["input_ids"].to('cuda')

# passing the prompt tokens to the generative model to generate the response tokens
with torch.no_grad():
    generation_output = base_model.generate(
        input_ids=input_ids,
        attention_mask=torch.ones_like(input_ids),
        generation_config=generation_config,
    )    
    
# decode the model generated tokens, and printing the generated output.
output_text = tokenizer.decode(generation_output[0].cuda(), skip_special_tokens=True).strip()
print(output_text)

# Calculate the elapsed time
elapsed_time = time.time() - start_time
print("Elapsed Time:", elapsed_time, "seconds")

Human: What do you know about the sun?
Assistant: The Sun is a G-type main sequence star located at near the center of the Solar System. It accounts for approximately 98%–99% (by mass) or roughly two thirds (by volume and luminosity in terms such as brightness, energy output or radius), with all other stars combined accounting for merely ~0.1 to 2% its total contribution; it forms part of an isolated binary system along with only one known companion that completes nearly exactly one orbit every 365 days, which itself takes almost precisely twice Jupiter's orbital period. Its proper motion relative to distant background objects results from its being carried around the galactic Center by the Milky Way’s gravitational pull within the local neighborhood where several dozen thousand neighboring nearby stars also move together much like beads strung on stringed necklaces wound up into compact spherical clusters..

The Sun has no permanent polar magnetic field because convection currents con

## Prompt 2 : Multi-Step Reasoning Question

In [38]:
query = "Prepare a plan how to make a cake, considering budget limitations, ingredient preference. Generate a step-by-step answer."

# Start the time counter
start_time = time.time()

inputs = tokenizer("Human: "+query+"\nAssistant:", return_tensors="pt")
input_ids = inputs["input_ids"].to('cuda')

# passing the prompt tokens to the generative model to generate the response tokens
with torch.no_grad():
    generation_output = base_model.generate(
        input_ids=input_ids,
        attention_mask=torch.ones_like(input_ids),
        generation_config=generation_config,
    )    
    
# decode the model generated tokens, and printing the generated output.
output_text = tokenizer.decode(generation_output[0].cuda(), skip_special_tokens=True).strip()
print(output_text)

# Calculate the elapsed time
elapsed_time = time.time() - start_time
print("Elapsed Time:", elapsed_time, "seconds")

Human: Prepare a plan how to make a cake, considering budget limitations, ingredient preference. Generate a step-by-step answer.
Assistant: Step 1: Make sure you have all the necessary ingredients and equipment needed for your cakes before starting with baking process so that any unforeseen situation can be handled carefully by having backup items such as extra milk or sugar in case of accidental spillage etc.. Also keep ready some amount butter/oil for greasing purpose if required and also preheat oven while making preparations like mixing bowl , whisk & spatula / spoon handy near stove top . In fact try keeping everything at reachable distance from place where cooking will take happen ! It not only saves time but makes things easy too because every minute counts when one is busy doing multiple tasks simultaneously ;).   Here are steps according preparation which takes less costly yet delicious recipes :     
Preparation Time –   20 min( includes chilling+mixing )Cooking Period -    3

## Prompt 3 : Real-Time Interaction Simulation

In [39]:
query = "Imagine you're a senior developer helping a junior developer fix a bug in a web app. Explain how to find and fix the problem step by step, like checking the code and trying out different solutions."
# Start the time counter
start_time = time.time()

inputs = tokenizer("Human: "+query+"\nAssistant:", return_tensors="pt")
input_ids = inputs["input_ids"].to('cuda')

# passing the prompt tokens to the generative model to generate the response tokens
with torch.no_grad():
    generation_output = base_model.generate(
        input_ids=input_ids,
        attention_mask=torch.ones_like(input_ids),
        generation_config=generation_config,
    )    
    
# decode the model generated tokens, and printing the generated output.
output_text = tokenizer.decode(generation_output[0].cuda(), skip_special_tokens=True).strip()
print(output_text)

# Calculate the elapsed time
elapsed_time = time.time() - start_time
print("Elapsed Time:", elapsed_time, "seconds")

Human: Imagine you're a senior developer helping a junior developer fix a bug in a web app. Explain how to find and fix the problem step by step, like checking the code and trying out different solutions.
Assistant: Here are some steps that can be followed as an outline for finding bugs or problems within your existing scripts/codes: 1) Reproduce it - Before anything else make sure this is happening consistently even outside of development mode if needed (ie; production vs dev version). Once reproduced reliably take note all variables at hand such as page load times, browser type & OS etc...2) Identify what specifically has gone wrong- Is there no data being returned? Wrong message coming up on screen after form submission instead of expected error handling?,3) Check every variable involved with scripting before execution begins then identify which part of script fails first / where exactly i.e., Javascript errors often come from syntax mistakes but also check whether everything seems 

# microsoft/phi-2.7B

In [40]:
# importing the packages
import os
import torch
import random
import string
#import pinecone
import warnings
#from datasets import load_dataset
from langchain import HuggingFacePipeline
from langchain.chains import RetrievalQA
from langchain.vectorstores import Pinecone
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

warnings.filterwarnings('ignore')

import time
import os, torch, logging
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, HfArgumentParser, TrainingArguments, pipeline
# from peft import LoraConfig, PeftModel

from transformers import BitsAndBytesConfig

# importing the packages
import torch
import warnings
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig

warnings.filterwarnings('ignore')

# Model and tokenizer names
#base_model_name = "mistralai/Mistral-7B-Instruct-v0.2"
#base_model_name = "CallComply/Starling-LM-11B-alpha"
base_model_name="microsoft/phi-2"

# Tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"  # Fix for fp16

# Quantization Config
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=False
)

# Model
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    quantization_config=quant_config,
    device_map="auto"
)
base_model.config.use_cache = False
base_model.config.pretraining_tp = 1

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Loading checkpoint shards: 100%|██████████| 2/2 [00:02<00:00,  1.13s/it]


In [41]:
# setting the model generation configuration
generation_config = GenerationConfig(
    do_sample = True,
    temperature = 0.8,
    repetition_penalty = 1.5,
    max_new_tokens = 256,
    pad_token_id = tokenizer.pad_token_id,
    eos_token_id = tokenizer.eos_token_id
)

## Prompt 1 : Simple Q/A

In [42]:
# inputs = tokenizer("Human: How old is the sun?\nAssistant:", return_tensors="pt")

query = "What do you know about the sun?"
# Start the time counter
start_time = time.time()

inputs = tokenizer("Human: "+query+"\nAssistant:", return_tensors="pt")
input_ids = inputs["input_ids"].to('cuda')

# passing the prompt tokens to the generative model to generate the response tokens
with torch.no_grad():
    generation_output = base_model.generate(
        input_ids=input_ids,
        attention_mask=torch.ones_like(input_ids),
        generation_config=generation_config,
    )    
    
# decode the model generated tokens, and printing the generated output.
output_text = tokenizer.decode(generation_output[0].cuda(), skip_special_tokens=True).strip()
print(output_text)

# Calculate the elapsed time
elapsed_time = time.time() - start_time
print("Elapsed Time:", elapsed_time, "seconds")

Human: What do you know about the sun?
Assistant: I have learned that ________.
Elapsed Time: 0.4977257251739502 seconds


## Prompt 2 : Multi-Step Reasoning Question

In [43]:
query = "Prepare a plan how to make a cake, considering budget limitations, ingredient preference. Generate a step-by-step answer."

# Start the time counter
start_time = time.time()

inputs = tokenizer("Human: "+query+"\nAssistant:", return_tensors="pt")
input_ids = inputs["input_ids"].to('cuda')

# passing the prompt tokens to the generative model to generate the response tokens
with torch.no_grad():
    generation_output = base_model.generate(
        input_ids=input_ids,
        attention_mask=torch.ones_like(input_ids),
        generation_config=generation_config,
    )    
    
# decode the model generated tokens, and printing the generated output.
output_text = tokenizer.decode(generation_output[0].cuda(), skip_special_tokens=True).strip()
print(output_text)

# Calculate the elapsed time
elapsed_time = time.time() - start_time
print("Elapsed Time:", elapsed_time, "seconds")

Human: Prepare a plan how to make a cake, considering budget limitations, ingredient preference. Generate a step-by-step answer.
Assistant: 1) Determine the size and type of pan needed for your desired serving quantity (e.g., 9x13 inch). 
2) Research recipes that fit within your preferred flavors/custard base ingredients & sugar amounts using online resources such as cookbooks or cooking websites like Allrecipes etc. This may involve comparing multiple recipe results before making final decisions on which ones you will use because each person has different preferences when it comes down with flavor variation's in their desserts(i think vanilla is popular but other people might prefer chocolate!). 3)? Add all necessary kitchen supplies into shopping list format so nothing gets overlooked during preparation time! 4?). Make sure everything fits comfortably under budget constraints without sacrificing quality by eliminating any unnecessary items; don't buy anything if there isn’t enough mo

## Prompt 3 : Real-Time Interaction Simulation

In [44]:
query = "Imagine you're a senior developer helping a junior developer fix a bug in a web app. Explain how to find and fix the problem step by step, like checking the code and trying out different solutions."
# Start the time counter
start_time = time.time()

inputs = tokenizer("Human: "+query+"\nAssistant:", return_tensors="pt")
input_ids = inputs["input_ids"].to('cuda')

# passing the prompt tokens to the generative model to generate the response tokens
with torch.no_grad():
    generation_output = base_model.generate(
        input_ids=input_ids,
        attention_mask=torch.ones_like(input_ids),
        generation_config=generation_config,
    )    
    
# decode the model generated tokens, and printing the generated output.
output_text = tokenizer.decode(generation_output[0].cuda(), skip_special_tokens=True).strip()
print(output_text)

# Calculate the elapsed time
elapsed_time = time.time() - start_time
print("Elapsed Time:", elapsed_time, "seconds")

Human: Imagine you're a senior developer helping a junior developer fix a bug in a web app. Explain how to find and fix the problem step by step, like checking the code and trying out different solutions.
Assistant: 1) Open up your editor or IDE where the error occurred  2️⃣ Find all relevant lines of codes within 10-15 minutes 3🖥 Test each line thoroughly for errors 4👆 Use online forums/resources when stuck 5😊 Review similar issues solved effectively on Stack Overflow 6❌ If nothing was found after 20 mins 🙏 Try running debugging tools 7&8™ Consult colleagues if needed 9–10 - Fixing bugs is an iterative process that requires patience but it's key part being able work through them efficiently! 11 – Share successful fixes with other developers 12 & 13— Keep track log files 14 — Reflect back 15° Finally enjoy what works well 😁
Elapsed Time: 6.6301305294036865 seconds
